Nagaraj K
1RVU22BSC059

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
class TeacherModel(nn.Module):
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 58.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.64MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 13.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 1.23MB/s]


In [ ]:
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher = F.softmax(teacher_logits / temperature, dim=1)
    soft_student = F.log_softmax(student_logits / temperature, dim=1)
    kl_div = F.kl_div(soft_student, soft_teacher) * (temperature ** 2)
    ce_loss = F.cross_entropy(student_logits, labels)
    return alpha * kl_div + (1 - alpha) * ce_loss

In [ ]:
teacher = TeacherModel()
optimizer = optim.Adam(teacher.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
teacher = TeacherModel()
optimizer = optim.Adam(teacher.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
def train_teacher(model, train_loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f"Teacher Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_teacher(teacher, train_loader, optimizer, criterion)

Teacher Epoch 1/5, Loss: 0.020427126437425613
Teacher Epoch 2/5, Loss: 0.16831161081790924
Teacher Epoch 3/5, Loss: 0.024650944396853447
Teacher Epoch 4/5, Loss: 0.052948590368032455
Teacher Epoch 5/5, Loss: 0.018121574074029922


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        # Define the layers of the student model here
        # This is a simple example, you can adjust the architecture as needed
        self.fc1 = nn.Linear(28*28, 300)  # Reduced number of neurons compared to Teacher
        self.fc2 = nn.Linear(300, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    return accuracy




def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [ ]:
teacher_params = count_parameters(teacher)
# Create an instance of the StudentModel
student = StudentModel()
student_params = count_parameters(student)

teacher_accuracy = evaluate(teacher, test_loader)
student_accuracy = evaluate(student, test_loader)

print(f"Teacher Model Parameters: {teacher_params}")
print(f"Student Model Parameters: {student_params}")

print(f"Teacher Accuracy: {teacher_accuracy:.2f}%, Parameters: {teacher_params}")
print(f"Student Accuracy: {student_accuracy:.2f}%, Parameters: {student_params}")

Teacher Model Parameters: 2395210
Student Model Parameters: 238510
Teacher Accuracy: 96.80%, Parameters: 2395210
Student Accuracy: 7.64%, Parameters: 238510
